In [85]:
from pathlib import Path

import polars as pl
import numpy as np

In [ ]:
# day 1.1

df = pl.read_csv(Path("inputs/day_1.csv"))
left = df.select("left").to_series().sort()
right = df.select("right").to_series().sort()
(left - right).abs().sum()

1506483

In [ ]:
# day 1.2

right_counts = df.group_by("right").len()  # tells us how many times each unique value in right appears
df.join(right_counts, left_on="left", right_on="right", how="left").select(
    [
        pl.col("left"),
        pl.col("right"),
        pl.col("len").fill_null(0).alias("count"),
    ]
).with_columns(score=pl.col("left") * pl.col("count")).select("score").sum()

score
i64
23126924


In [ ]:
# day 2.1

path = Path("inputs/day_2.txt")
lines = path.read_text().splitlines()

def is_safe(nums: list[int]) -> bool:
    nums = np.array(nums)
    diff = np.diff(nums)
    in_range = np.all(np.abs(diff) >= 1) and np.all(np.abs(diff) <= 3)
    consistent_sign = np.all(diff < 0) or np.all(diff > 0)
    return in_range and consistent_sign

num_safe = 0
for line in lines:
    report = list(map(int, line.split()))
    diff = np.diff(report)
    if is_safe(report):
        num_safe += 1

num_safe

341

In [ ]:
# day 2.1 polars

MAX_COLUMNS = 8
path = Path("inputs/day_2.txt")
schema = {f"col{i}": pl.Int64 for i in range(MAX_COLUMNS)}
df = pl.read_csv(path, separator=" ", schema=schema, has_header=False)

# partition based on which one should be increasing
increasing_df = df.filter(pl.col("col2") > pl.col("col1"))
decreasing_df = df.filter(pl.col("col2") < pl.col("col1"))

# define the filter expressions for each case (increasing/decreasing)
def get_filter_expression(increasing: bool, column_idx: int):
    increment_expression = (
        (pl.col(f"col{column_idx}") < pl.col(f"col{column_idx+1}"))
        if increasing else (pl.col(f"col{column_idx}") > pl.col(f"col{column_idx+1}"))
    )
    return (
        (abs(pl.col(f"col{i}").sub(pl.col(f"col{i+1}"))) >= 1)
        & (abs(pl.col(f"col{i}").sub(pl.col(f"col{i+1}"))) <= 3)
        & increment_expression
        | (pl.col(f"col{i}").is_null())
        | (pl.col(f"col{i+1}").is_null())
    )

# filter dataframes based on condition
for i in range(len(df.columns) - 1):
    increasing_df = increasing_df.filter(get_filter_expression(True, i))
    decreasing_df = decreasing_df.filter(get_filter_expression(False, i))

len(increasing_df) + len(decreasing_df)

341

In [ ]:
# day 2.2

path = Path("inputs/day_2.txt")
lines = path.read_text().splitlines()

num_safe = 0
for line in lines:
    report = list(map(int, line.split()))
    if is_safe(report):
        num_safe += 1
    else:
        for i in range(len(report)):
            sub_report = report[:i] + report[i+1:]
            if is_safe(sub_report):
                num_safe += 1
                break

num_safe


404

In [ ]:
# day 3.1

import re

path = Path("inputs/day_3.txt")
with Path(path).open("r") as file:
    content = file.read()

pattern = r"mul\(\s*([+-]?\d+)\s*,\s*([+-]?\d+)\s*\)"
matches = re.findall(pattern, content)
mult_sum = sum([int(match[0]) * int(match[1]) for match in matches])
mult_sum

166905464

In [334]:
# day 3.2

import re

path = Path("inputs/day_3.txt")
with Path(path).open("r") as file:
    content = file.read()

pattern = r"mul\(\s*([+-]?\d+)\s*,\s*([+-]?\d+)\s*\)|(do\(\)|don't\(\))"
matches = re.findall(pattern, content)

dont_str = "don't()"
do_str = "do()"
do = True
mult_sum = 0

for group in matches:
    if dont_str in group:
        do = False

    if do_str in group:
        do = True

    if do and group[0] != "" and group[1] != "":
        mult_sum += int(group[0]) * int(group[1])

mult_sum

72948684

In [ ]:
# day 4.1

contents = np.array([list(line.strip()) for line in list(Path("inputs/day_4.txt").read_text().splitlines())])

target = "XMAS"

pad = len(target) - 1
contents = np.pad(contents, pad, mode="constant", constant_values="-")

window_length = 2 * (len(target)) - 1

directions = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]

target_matches = 0
for row_idx in range(contents.shape[0]):
    for column_idx in range(contents.shape[1]):
        # find all possible starting locations of the target string
        if contents[row_idx][column_idx] == target[0]:
            # get the window for the possible target string
            columns = np.arange(column_idx - len(target) + 1, column_idx + len(target))
            column_indices = np.repeat(columns, window_length).reshape((window_length, window_length)).T
            rows = np.arange(row_idx - len(target) + 1, row_idx + len(target))
            row_indices = np.tile(rows, window_length).reshape((window_length, window_length)).T
            window = contents[row_indices, column_indices]

            # determine the number of times the target string occurs in the window
            center = len(window) // 2
            for row_direction, column_direction in directions:
                char_matches = 0
                for i in range(len(target)):
                    r, c = center + row_direction * i, center + column_direction * i
                    if window[r, c] != target[i]:
                        break
                    else:
                        char_matches += 1

                if char_matches == len(target):
                    target_matches += 1

target_matches

2530